# 極小場域定位

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import sys, os
import pandas as pd
import matplotlib.pyplot as plt

from csidataset import *
import data_loader
from data_loader import *
sys.path.append("/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool")
import denoise
from model import *
import time

In [2]:
base_path = "/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/csi_dataset/localization_phone/micro/0303/80Mhz/csv"

In [3]:
reference_points = {
    f"{base_path}/reference_point{i}.xlsx": i for i in range(1, 37)
}

# 確認字典內容
print(reference_points)



{'/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/csi_dataset/localization_phone/micro/0303/80Mhz/csv/reference_point1.xlsx': 1, '/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/csi_dataset/localization_phone/micro/0303/80Mhz/csv/reference_point2.xlsx': 2, '/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/csi_dataset/localization_phone/micro/0303/80Mhz/csv/reference_point3.xlsx': 3, '/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/csi_dataset/localization_phone/micro/0303/80Mhz/csv/reference_point4.xlsx': 4, '/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/csi_dataset/localization_phone/micro/0303/80Mhz/csv/reference_point5.xlsx': 5, '/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/csi_dataset/localization_phone/micro/0303/80Mhz/csv/reference_point6.xlsx': 6, '/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/csi_dataset/localization_phone/micro/0303/80Mhz/csv/reference_point7.xlsx': 7, '/med

In [4]:
def load_data(reference_points):
    data = []          
    rp_labels = []     

    for path, ref_id in reference_points.items():
        df = pd.read_excel(path)
        data.append(df.values)
        rp_labels.extend([ref_id] * len(df))  # 只保留 Reference Point ID

    data = pd.DataFrame(np.vstack(data))
    rp_labels = pd.Series(rp_labels, name="Reference Point ID")  # 轉為 Pandas Series

    return data, rp_labels

In [5]:
data, rp_labels = load_data(reference_points)

In [6]:

print(data.shape)       # 顯示數據
print(rp_labels.shape)  # 顯示 Reference Point ID

(7164, 470)
(7164,)


In [7]:
amp_data = np.array(data.iloc[:, :234])
phase_data = np.array(data.iloc[:, 234:-2])  

In [8]:
amp_d = denoise.preprocess_csi_for_fingerprint2(amp_data)
phase_d = denoise.preprocess_csi_for_fingerprint2(phase_data)

#amp_phase_d = np.concatenate((amp_d, phase_d), axis=1)
#print(amp_phase_d.shape)

In [9]:
encoder = OneHotEncoder(sparse_output=False)
one_hot_labels = encoder.fit_transform(np.array(rp_labels).reshape(-1, 1))

In [10]:
print(amp_d.shape)
print(phase_data.shape)
print(one_hot_labels.shape)

(7164, 234)
(7164, 234)
(7164, 36)


In [11]:
amp_train, amp_temp, y_train, y_temp = train_test_split(amp_d, one_hot_labels, test_size=0.3, random_state=42)
amp_val, amp_test, y_val, y_test = train_test_split(amp_temp, y_temp, test_size=1/3, random_state=42)

In [12]:
amp_train.shape

(5014, 234)

In [13]:
batch_size = 32

train_dataset = CSIDataset(amp_train, y_train)
val_dataset = CSIDataset(amp_val, y_val)
test_dataset = CSIDataset(amp_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [14]:
import torch.nn as nn
import torch.optim as optim

# 初始化模型
maxlen = amp_d.shape[1]  # 序列長度
embed_dim = 128
num_heads = 4
ff_dim = 128
num_classes = one_hot_labels.shape[1]
num_layers = 1
dropout_rate = 0.1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerClassifier(maxlen, embed_dim, num_heads, ff_dim, num_classes, num_layers, dropout_rate).to(device)

# 損失函數
criterion = nn.CrossEntropyLoss()

# 優化器
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# 學習率調整器
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=15, verbose=True)

/home/mcs/anaconda3/envs/kyle_ai/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [15]:
from torchsummary import summary

summary(model, input_size=(234, 1))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Embedding-1             [-1, 234, 128]          29,952
            Linear-2             [-1, 234, 128]             256
TokenAndPositionEmbedding-3             [-1, 234, 128]               0
MultiheadAttention-4  [[-1, 234, 128], [-1, 234, 234]]               0
           Dropout-5             [-1, 234, 128]               0
         LayerNorm-6             [-1, 234, 128]             256
            Linear-7             [-1, 234, 128]          16,512
              ReLU-8             [-1, 234, 128]               0
            Linear-9             [-1, 234, 128]          16,512
          Dropout-10             [-1, 234, 128]               0
        LayerNorm-11             [-1, 234, 128]             256
 TransformerBlock-12             [-1, 234, 128]               0
AdaptiveAvgPool1d-13               [-1, 128, 1]               0
           Linear-14     

In [16]:
import torch
import matplotlib.pyplot as plt

# 儲存最佳模型
best_val_loss = float('inf')
best_model_path = "best_model_mirco.pth"

# 訓練參數
epochs = 200

# early stop
patience = 20
counter = 0  

# 訓練過程中的 loss 和 accuracy
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(epochs):
    # ---- 訓練階段 ----
    start_time = time.perf_counter()
    model.train()
    train_loss = 0.0
    train_correct = 0
    total_train = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        # CrossEntropyLoss 需要 class index
        loss = criterion(outputs, torch.argmax(labels, dim=1))
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        train_correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

    avg_train_loss = train_loss / len(train_loader.dataset)
    train_acc = 100 * train_correct / total_train

    # ----驗證階段----
    model.eval()
    val_loss = 0.0
    val_correct = 0
    total_val = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, torch.argmax(labels, dim=1))

            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            val_correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

    avg_val_loss = val_loss / len(val_loader.dataset)
    val_acc = 100 * val_correct / total_val

    end_time = time.perf_counter()
    epoch_time = end_time - start_time

    # 紀錄每個 epoch 的 loss 和 accuracy
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)


    # 輸出當前 epoch 的結果
    print(f"Epoch [{epoch+1}/{epochs}] | "
          f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.2f}% | time: {epoch_time:.2f} s")

    # ---- 儲存最佳模型 ----
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), best_model_path)
        print(f"✅ 儲存最佳模型 (Val Loss: {best_val_loss:.4f}) 至 {best_model_path}")
        counter = 0
    else:
        counter += 1

    if counter >= patience:
        print(f"Early stop at epoch {epoch+1}")
        break

print("訓練完成！")


Epoch [1/200] | Train Loss: 3.1588 | Train Acc: 10.35% | Val Loss: 2.4013 | Val Acc: 25.47% | time: 0.63 s
✅ 儲存最佳模型 (Val Loss: 2.4013) 至 best_model_mirco.pth
Epoch [2/200] | Train Loss: 1.7396 | Train Acc: 45.25% | Val Loss: 1.1278 | Val Acc: 70.90% | time: 0.56 s
✅ 儲存最佳模型 (Val Loss: 1.1278) 至 best_model_mirco.pth
Epoch [3/200] | Train Loss: 0.8944 | Train Acc: 74.59% | Val Loss: 0.6023 | Val Acc: 84.30% | time: 0.56 s
✅ 儲存最佳模型 (Val Loss: 0.6023) 至 best_model_mirco.pth
Epoch [4/200] | Train Loss: 0.6097 | Train Acc: 82.27% | Val Loss: 0.5472 | Val Acc: 84.44% | time: 0.56 s
✅ 儲存最佳模型 (Val Loss: 0.5472) 至 best_model_mirco.pth
Epoch [5/200] | Train Loss: 0.4549 | Train Acc: 86.94% | Val Loss: 0.4149 | Val Acc: 89.32% | time: 0.56 s
✅ 儲存最佳模型 (Val Loss: 0.4149) 至 best_model_mirco.pth
Epoch [6/200] | Train Loss: 0.3988 | Train Acc: 87.55% | Val Loss: 0.3756 | Val Acc: 90.23% | time: 0.56 s
✅ 儲存最佳模型 (Val Loss: 0.3756) 至 best_model_mirco.pth
Epoch [7/200] | Train Loss: 0.3707 | Train Acc: 87.8